# Aritmética en la computadora 1

Los *números de punto flotante* son una aproximación de los números reales en la computadora. El ejemplo más usado, mas no el único, corresponde a los `Float64`.

La idea de este notebook es entender qué son los números de punto flotante, y concientizarnos de que el conjunto de los números de punto flotante no corresponde a $\mathbb{R}$. De pasada, ensayaremos crear estructuras moldeadas a nuestro gusto/necesidad.

# Números enteros

Los humanos solemos utilizar base 10 para representar los números.

**[1]** ¿Por qué crees que nos es natural usar base 10?

**[2]** 

(i) Explorando la función `bits()` usada con enteros, ¿cómo se representa en binario el número entero más grande de 64 bits? Escríbelo en base 10 y comprueba el resultado.

(ii) ¿Y el número entero más pequeño?

(iii) ¿Qué ocurre si al más grande le sumamos 1? ¿Qué sentido el resultado?

(iv) Checa tus reasultados con las funciones `typemax` y `typemin`

# Números reales

El problema obvio en representar los números reales es que éstos son infinitos no numerables, y tenemos recursos (memoria y tiempo) finitos.

## Aritmética de punto fijo

Una alternativa sencilla para representar números reales es, simplemente, considerar *algunos* racionales en el intervalo $[0,1)$, y usar cocientes entre dos números enteros positivos. Para explotar al máximo los 64-bits, podemos usar `n :: UInt64` como el numerador, y `m = typemax(UInt64)` como denominador, para esto. Estos son los llamados *números de punto fijo*.

Esto, de facto, permite escribir un número $x\in [0,1)$ como

$$
x = \frac{n}{m} = (0.a_1\,a_2\ldots a_n)_{(2)} = \sum_{i=1}^n a_i \, 2^{-i},
$$

donde todo $a_i \in \{0, 1\}$.

**[3]**

(i) Si `x` es un `Float64`, escribe la función `nptofijo(x::Float64)` que permita representar a `x` como un `UInt64` que equivale al número de punto fijo.

(ii) Si `y :: UInt64` es un número de punto fijo, su representación como un `Float64` será simplemente `y / typemax(UInt64)`. Muestra que la función escrita en (i) es consistente.

(iii) Dado que la aritmética para `UInt64` está definida, verifica la consistencia de algunas operaciones.

(iv) ¿Qué limitación tiene esta manera de representar los números?

## Representación de los números reales

Cuaquier número real $x\in \mathbb{R}$ lo podemos *representar* como

$$
x = (-1)^\sigma \sum_{k=-\infty}^n b_k \,\beta^k, 
$$

donde $0 \le b_k \le \beta -1$, para toda k. Aquí, $\sigma=0,1$ define el signo del número en cuestión, y $\beta=2, 3, ...$ es un entero mayor que 1 que define la *base* en que representamos el número (e.g., base 2, base 8, base 10, etc).


Las sutilezas usuales de los ceros a la izquierda, que simplemente se omiten, o las colas infinitas de $\beta-1$ se aplican aquí, como por ejemplo $0.1999999\dots_{(10)}=0.2_{(10)}$. Esta última se soluciona pidiendo que (a) $0 \le b_k \le \beta -1$, y (b) $0 \le b_k \le \beta -2$ para un número infinito de $k$.

## Números de punto flotante

Una representación conveniente de los números reales en la computadora es la siguiente:

$$
x = (-1)^\sigma \cdot s \cdot \beta^r, 
$$

donde $\sigma$ define el signo nuevamente, $\beta$ es la base de la representación, $r$ es un exponente (número entero) y $s$, el *significante* (o mantisa), es un número real positivo (o cero).

El significante es un número (real y positivo) que por convención representamos como
$$
s = b_0 . b_{-1} b_{-2} b_{-3} \dots,
$$
donde cada "dígito" $b_{k}$ cumple $ 0\le b_k \le \beta - 1$ para todo $k$, y $ 0\le b_k \le \beta - 2$ para infinitas $k$.

Ejemplos:

\begin{eqnarray}
-0.5 = (-1)^1 \cdot 1.0 \cdot 2^{-1} = 0.1_{(2)} & \quad \Longrightarrow \quad&  \sigma = 1, \beta=2, r = -1, s = 1.0,\\
2/3 = (-1)^0 \cdot 2.0 \cdot 3^{-1} = 0.2_{(3)}& \quad \Longrightarrow \quad&  \sigma = 0, \beta=3, r = -1, s = 2.0,\\
30.1 = (-1)^0 \cdot 0.301 \cdot 10^{2} & \quad \Longrightarrow \quad&  \sigma = 0, \beta=10, r = 2, s = 0.301
\end{eqnarray}

**[4]** La representación descrita arriba es *obviamente* **no** única e introduce otro tipo de redundancias. Da dos ejemplos de representaciones distintas de un mismo número en base 10.

**[5]** Escribe la representación de $0.1_{(10)}$ en binario. (Procura no restringirte a 64 bits!) Restringiéndonos a 64 bits, como interpretas el resultado que arroja `BigFloat(0.1)`.

Una manera de resolver el problema de la representación única es *imponer* que el significante $s$ (distinto de cero) sea tal que $b_0 \neq 0$. De esta manera, el último ejemplo quedaría:

$$
30.1 = (-1)^0 \cdot 3.01 \cdot 10^{1} \quad \Longrightarrow \quad  \sigma = 0, \beta=10, r = 1, s = 3.01
$$

Esto define al conjunto de números *normales* de punto flotante en base $\beta$, $\mathbb{F}_\beta$.

**[6]** El cero, como siempre, es especial. ¿Por qué? ¿Cuál es una convención adecuada para representarlo?

## Representación en la computadora

En la computadora, no podemos representar **todos** los números reales: se trata de un conjunto no numerable, y la memoria es finita.

Debemos entonces limitar la precisión con la que trabajamos, por ejemplo, limitando el número de dígitos significativos de la mantisa. Esto genera el conjunto de elementos $\mathbb{F}_{\beta,p}$ cuyo significante escribiremos como

$$
s = b_0 . b_{1} b_{2} b_{3} \dots b_{p-1},
$$

donde $b_0\neq 0$, y $0 \le b_i \le \beta-1$ para $i=1,..., p-1$. El entero $p$ es la precisión.

**[7]** ¿Por qué es cierta la afirmación de que $\mathbb{F}_{\beta,p}$ es un conjunto *infinito* numerable?

Por ejemplo, los `Float64` usan 64 bits, están definidos en base $\beta=2$, y los bits disponibles se ordenan de la siguiente manera:

- 1 bit: $\sigma$
- 11 bits: $r$
- 52 bits: $s$

NOTA: Dado que los números de punto flotante se definen con $b_0\ne 0$, en el caso de $\beta =2$ tenemos que $b_0=1$ para estos números. Esto permite omitir este bit, y ganar uno más. Es por eso que `Float64` corresponde a 53-bits.

Una manera de generar un número finito de elementos (¡tenemos memoria finita en la computadora!) es imponiendo *además* cotas para los posibles exponentes, es decir, $r_{min} \le r \le r_{max}$.

**[8]** 

(i) Usando la representación en base 2 de `1.0` y `0.5` exacta, `bits(1.0)`, `bits(0.5)`, `parse(Int, x, 2)` y un poco de intuición(!), obtén una definición de la implementación de $r$ en la computadora.

(ii) ¿Cuales son las cotas para $r$ en `Float64`?

(iii) ¿Cuántos bits se usan para $r$ para `Float32` y cuales son las cotas?

# `MiNormFloat`

**[9]** 

(i) Define las *constantes* `prec = 3`, `rmin = -1`, `rmax = 2`. (Ser constante en julia significa que el tipo no puede cambiar, aunque posiblemente puede cambiar el valor; ver la ayuda para `const`.)


(ii) Define la estructura `MiNormFloat` cuyos campos sean:
- `base :: Int`, la base ($\beta\ge 2$),

- `sigma :: Bool`, el signo (nota que `Int(true) = 1` y `Int(false)=0`),

- `expo :: Int`, el exponente (que debe cumplir `rmin ≤ expo ≤ rmax`, con `rmin` y `rmax` los enteros fijos que definiste,

- `mant :: NTuple{prec, Int}`, donde `prec` es el entero (pequeño) que definiste como precisión.

El tipo `MiNormFloat` debe tener un constructor interno para asegurar que **todas** las restricciones de los números de punto fijo se cumplan. Cualquier `MiNormFloat` debe corresponder a un número de punto flotante normal bien definido, o al cero. (Dado que la base es arbitraria, no podemos explotar el bit que se puede omitir si $\beta=2$.)

(iii) Define varias pruebas y muestra que todo funciona adecuadamente.

**[10]** Escribe la función que convierta un número de tipo `MiNormFloat` a uno `Float64`. Define varias pruebas que muestren que la implementación es correcta.

**[11]** 

(i) Calcula *analíticamente* el número de elementos que tiene el conjunto de números de punto flotante normales, dados la precisión `p`, y los exponentes `rmin` y `rmax` (¡para cualquier $\beta$!).

(ii) ¿Cuales son los números reales que describe el conjunto de números de punto flotante normales para `prec = 3`, `rmin = -1` y `rmax = 2` con $\beta =2$ ? 

(iii) ¿Es homogéneo el espaciamiento entre los números de punto flotantes obtenidos en (ii)?

(iv) Dibuja en una recta *todos* los números normales de punto flotante que definen estos parámetros, incluyendo el cero. ¿Cómo es el espaciamiento en torno al cero?

**[12]** La suma de elementos tipo `Mifloat`, ¿da siempre un elemento tipo `Mifloat`? Argumenta con uno o varios ejemplos.

# Números subnormales

Una manera de hacer que el espaciamiento entre el 0 y el primer número de punto flotante normal no sea tan grande, es introduciendo los llamados *números subnormales*. Éstos se definen permitiendo que $b_0=0$ únicamente con $r = r_{min}$.

**[13]** 

(i) Crea el tipo `MiFloat` que incluya a los números de punto flotante normales y subnormales dados `prec`, `rmin` y `rmax`.

(ii) Para `prec=3`, `rmin=-1`, `rmax=2` dibuja todos los números de punto flotante. ¿Como es el espaciamiento ahora cerca del cero?

(iii) ¿Es cerrada la suma del conjunto de números de punto flotante normales y subnormales?


# Resumen

**[14]** Escribe un resumen de lo visto en este notebook.